<a href="https://colab.research.google.com/github/AbrahamKong/-CMPE257-NLP_Poem_Part_2/blob/main/NLP_Poem_Scraping_9b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 

#NLP Part 1 : Poetry Analysis

*   NLP Basics
*   NER, POS, vectorization / embeddings
*   Word2Vec, Glove, Cove, FastText, etc.
*.  Similarity of words, context




1. NLP Data. Pick 2 poets, scrape 10 poems each . Put them in a json datastructure and save it under the poets name, in your team drive under folder 'DataSets' in the drive shared with you. 

2. CLAIM YOUR POETS!!! Note: your team has  to claim their 2 poets by entering in their names in this spreadsheet with a timestamp of when you entered this: no duplicate poets allowed !
https://docs.google.com/spreadsheets/d/1-KQVrGrbz5PvLhJNXhWm55WPIpmJPvsQQ2r9U-ZieGw/edit#gid=0

---for each poem of each poet:
3. Run POS and extract verbs, nouns and adjectives (use notebook given as an example)

4. Summarize each poem 

5. Store the summary, POS data in your json poet data structure and save to disk.

6. Interchange the verbs and adjectives of the two poets based on similar words in the other poets vocabulary of verbs and adjectives

6. a. save the data structure for the new poems:  poet1-poet2-poem1.json (replace poet1 adjectives and verbs with poet2's ) and poet2-poet1-poem1.json 

6. b. print out the new poems and save them to disk poet1-poet2.txt and poet2-poet1.txt [these files will contain 10 poems each)].
remember to measure and compare the cohesion and perplexity of the original poems and the new poems you generate !!

7. Combine poet1's and poet2's poem text and summarize it using the transformer

8. Run topic modeling on the combined text of each poet: what are the top 5 topics they are writing about?

9. Name the topics: use the new notebook : name_your_topic_background.ipynb (Links to an external site.) , to try and find the closest word that describes topic 0, topic 1, topic 2 : instead of topic 0, what is the closest word in that cluster that describes the topic?

# Claim 2 poets and Scrape 10 poems each
John Keats
Alfred Tennyson    

Scraping example colab: https://colab.research.google.com/drive/1Px_UggyRiQJzrIxvLtitz-Es0aZ5Au8b?authuser=3#scrollTo=3aTGAo7rMvjN&uniqifier=1



In [1]:
!pip install nltk
!pip install gensim
!pip install transformers
!pip install sentence-transformers
!pip install pyLDAvis
!pip install spacy

In [2]:
!pip install contractions

In [3]:
!pip install -U pandas-profiling

In [4]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import wordcloud
import nltk
import unicodedata
import contractions
import pickle
import urllib.request
import requests
import spacy
import json
%matplotlib inline

from bs4 import BeautifulSoup
from bs4.element import Comment
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from collections import defaultdict
from contractions import contractions_dict
from wordcloud import WordCloud, STOPWORDS
from transformers import pipeline

from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem.porter import PorterStemmer
from sentence_transformers import SentenceTransformer, util
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from spacy import displacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('punkt')

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from contractions import contractions_dict
from scipy import stats
from scipy import sparse

import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
scaler = MinMaxScaler()
std_scaler = StandardScaler()
stops = set(stopword_list)
stemmer = nltk.stem.SnowballStemmer('english')
wordlemmatizer = WordNetLemmatizer()
tokenizer = nltk.word_tokenize

ModuleNotFoundError: ignored

# NLP Data. Pick 2 poets, scrape 10 poems each . Put them in a json datastructure and save it under the poets name, in your team drive under folder 'DataSets'

## Get Scraped Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# df_john = pd.read_csv('/content/drive/MyDrive/DataSets/john_keats.csv', sep=';')
lh_df = pd.read_csv('https://raw.githubusercontent.com/AbrahamKong/-CMPE257-NLP_Poem_Part_2/main/langston_hughes.csv')

In [ ]:
lh_df.info()

In [ ]:
# df_alfred = pd.read_csv('/content/drive/MyDrive/DataSets/alfred_tennyson.csv', sep=';')
mjo_df = pd.read_csv('https://raw.githubusercontent.com/AbrahamKong/-CMPE257-NLP_Poem_Part_2/main/mary_j_oliver.csv')

In [ ]:
mjo_df.info()

## JSON Conversion

In [ ]:
result_lh = lh_df.to_json(orient="index")
parsed_lh = json.loads(result_lh)
json.dumps(parsed_lh, indent=4)

In [ ]:
# with open('/content/drive/MyDrive/DataSets/john_keats_final.txt', 'w') as outfile:
#     json.dump(result_john, outfile)
with open('/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/Homework 9b/Datasets/langston_hughes.csv', 'w') as outfile:
    json.dump(result_lh, outfile)

In [ ]:
result_mjo = mjo_df.to_json(orient="index")
parsed_mjo = json.loads(result_mjo)
json.dumps(parsed_mjo, indent=4)

In [ ]:
# with open('/content/drive/MyDrive/DataSets/alfred_tennyson_final.txt', 'w') as outfile:
#     json.dump(result_alfred, outfile)
with open('/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/Homework 9b/Datasets/mary_j_oliver.csv', 'w') as outfile:
    json.dump(result_mjo, outfile)

In [ ]:
parsed_mjo["0"]

## Build JSON Structure

In [ ]:
langston_hughes = {

}

mary_j_oliver = {

}

## Data Cleaning

In [ ]:
def cleanPoemText(text):
    

    text = contractions.fix(text)
    
    text = text.strip().lower()
    
    text = re.sub(r"'", ' ', text)
    
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    text = ' '.join([w for w in text.split()])

    # Replace multiple space with one space
    text = re.sub(' +', ' ', text)
    
    text = ''.join(text)

    return text

## Clean Poem Text

In [ ]:
embedding = ''
def buildPoemArray(poet, data):
  poems = []
  for i in data:
    poem = data[i]["Poem"]

    clean_poem = cleanPoemText(poem)
    idx = "poem" + str(i)
    poet[idx] = clean_poem
    poems.append(idx)
  poet['poems_array'] = poems


In [ ]:
buildPoemArray(langston_hughes, parsed_lh)
buildPoemArray(mary_j_oliver, parsed_mjo)

In [ ]:
langston_hughes

In [ ]:
mary_j_oliver

In [ ]:
langston_hughes_poems = langston_hughes.copy()
mary_j_oliver_poems = mary_j_oliver.copy()

# Run Part-Of-Speech, extract verbs, nouns and adjectives

In [ ]:
def getPOS(poet):
  global_vrb= set()
  global_nns= set()
  global_adj= set()
  for key in poet['poems_array']:
    text = poet[key]
    wrd_list = word_tokenize(text)
    pos_val = nltk.pos_tag(wrd_list)
    poet["pos_"+key] = pos_val

    vrb = set([word for (word, pos) in pos_val if (pos.startswith('VB'))])
    nns = set([word for (word, pos) in pos_val if (pos.startswith('NN'))])
    adj = set([word for (word, pos) in pos_val if (pos.startswith('JJ'))])

    poet["verbs_"+key] = list(vrb)
    poet["adjectives_"+key] = list(adj)
    poet["nouns_"+key] = list(nns)

    global_vrb = set.union(global_vrb, vrb)
    global_nns = set.union(global_nns, nns)
    global_adj = set.union(global_adj, adj)

  poet['all_verbs'] = list(global_vrb)
  poet['all_nouns'] = list(global_nns)
  poet['all_adjectives'] = list(global_adj)

In [ ]:
langston_hughes_pos = langston_hughes.copy()
getPOS(langston_hughes_pos)
langston_hughes_pos

In [ ]:
langston_hughes

In [ ]:
mary_j_oliver_pos = mary_j_oliver.copy()
getPOS(mary_j_oliver_pos)
mary_j_oliver_pos

In [ ]:
mary_j_oliver

# Store the POS JSON Data to File

In [ ]:
with open('/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/Homework 9b/Datasets/langston_hughes_pos_final.json', 'w') as outfile:
    json.dump(langston_hughes_pos, outfile)

In [ ]:
with open('/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/Homework 9b/Datasets/mary_j_oliver_pos_final.json', 'w') as outfile:
    json.dump(mary_j_oliver_pos, outfile)

# Summarization using transformers

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

In [ ]:
def getSummaryOfPoem(poet):
  def removeQuotes(mydata):
    new_data = mydata.strip('“”')
    return new_data
  count = 0
  for key in poet['poems_array']:
    count += 1;
    text = poet[key] 
    text = removeQuotes(text)
    summary_text = summarizer(text, max_length=512, min_length=5, do_sample=False)[0]['summary_text']
    poet["summary_"+key] = summary_text
  
  return poet

In [ ]:
from time import gmtime, strftime
print (strftime("%Y-%m-%d %H:%M:%S", gmtime()))

from time import gmtime, strftime
print (strftime("%Y-%m-%d %H:%M:%S", gmtime()))

import timeit

start = timeit.default_timer()

In [ ]:
langston_hughes_sum = langston_hughes.copy()
mary_j_oliver_sum = mary_j_oliver.copy()
data_langston_hughes = getSummaryOfPoem(langston_hughes_sum)
data_mary_j_oliver = getSummaryOfPoem(mary_j_oliver_sum)

In [ ]:


#Your statements here

stop = timeit.default_timer()

print('Time: ', stop - start)  

In [ ]:
langston_hughes_sum

In [ ]:
mary_j_oliver_sum

In [ ]:
data_mary_j_oliver

# Store the summarized JSON Data to File

In [ ]:
with open('/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/Homework 9b/Datasets/langston_hughes_summary_final.json', 'w') as outfile:
    json.dump(langston_hughes_sum, outfile)

In [ ]:
with open('/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/Homework 9b/Datasets/mary_j_oliver_summary_final.json', 'w') as outfile:
    json.dump(mary_j_oliver_sum, outfile)

# Interchange the Verbs and Adjectives for both the poets using transformer

In [ ]:
model = SentenceTransformer('stsb-roberta-large')

In [ ]:
def getSemanticSimilarity(p1,p2):
  embedding1 = model.encode(p1, convert_to_tensor=True)
  embedding2 = model.encode(p2, convert_to_tensor=True)
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
  print(cosine_scores) 
  similarity_score = cosine_scores.item()
  return similarity_score

In [ ]:
getSemanticSimilarity(langston_hughes_pos['all_verbs'][0],mary_j_oliver_pos["all_verbs"][0])

# Swapping the Verbs based on similarity score

In [ ]:

def swapVerbs(p1,p2,text):
    for v1 in p1:
      for v2 in p2:
        score = getSemanticSimilarity(v1,v2)
        if score > 0.3:
          text = text.replace(v1,v2)
          print("verb")
          print(text)
          break
    return text

# Swapping the Adjectives based on similarity score

In [ ]:
def swapAdjectives(p1,p2, poem):
  text = poem
  for a1 in p1:
    for a2 in p2:
      print("inside loop 2")
      print(a1)
      print(a2)
      print(text)
      score = getSemanticSimilarity(a1,a2)
      if score > 0.3:
        text = text.replace(a1,a2)
        print("adjective")
        print(text)
  return text

In [ ]:
def swapIteration(poet1, poet2, text):
    p2_verbs = poet2["all_verbs"]
    p1_verbs = poet1["all_verbs"]
    p2_verbs = p2_verbs[0:2]
    p1_verbs = p1_verbs[0:2]
    print(p1_verbs)
    print(p2_verbs)
    new_text = swapVerbs(p1_verbs, p2_verbs, text)
    print("new text verbs")
    print(new_text)
    # print(text)
    p2_adjectives = poet2["all_adjectives"]
    p1_adjectives = poet1["all_adjectives"]
    p2_adjectives = p2_adjectives[0:2]
    p1_adjectives = p1_adjectives[0:2]
    brand_new_text = swapAdjectives(p1_adjectives, p2_adjectives, new_text)
    print("new text adjectives")
    print(brand_new_text)
    return brand_new_text

John Poems Swapped Verbs and Adjectives of Alfred Tennyson Poems

In [ ]:
langston_hughes_poems_2 = langston_hughes_poems.copy()
langston_hughes_poems_2.pop('poems_array')
langston_hughes_poems_swapped=[]
for i in langston_hughes_poems:
  print("poems:")
  print(i)
  # inx = 'poem'+str(i);
  poem_str = str(langston_hughes_poems[i]);
  print(poem_str)

  langston_hughes_poems_swapped.append(swapIteration(langston_hughes_pos, mary_j_oliver_pos, poem_str))


# Printing the new poems after swapping

In [ ]:
langston_hughes_poems_swapped

## TODO: We need to format the poem in a human readable manner

 Alfred Tennyson Poems Swapped Verbs  and Adjectives of John Poems


In [ ]:
mary_j_oliver_poems_swapped = []
mary_j_oliver_poems_2 = mary_j_oliver_poems.copy()
mary_j_oliver_poems_2.pop('poems_array')
for i in mary_j_oliver_poems:
  print("poems:")
  print(i)
  # inx = 'poem'+str(i);
  poem_str_2 = str(mary_j_oliver_poems[i]);
  print(poem_str_2)

  mary_j_oliver_poems_swapped.append(swapIteration(langston_hughes_pos, mary_j_oliver_pos, poem_str_2))


# Printing the new poems after swapping

In [ ]:
mary_j_oliver_poems_swapped

# Swapped Poem

In [ ]:
def buildSwappedPoemArray(poet,data):
  poems = []
  data_length = len(data)
  for i in range(data_length):
    poem = data[i]
    #clean_poem = cleanPoemText(poem)
    idx = "poem" + str(i)
    poet[idx] = poem
    poems.append(idx)
  poet['poems_swapped_array'] = poems

In [ ]:
langston_hughes_swapped = {

}

mary_j_oliver_swapped = {

}


In [ ]:
langston_hughes_poems_swapped


In [ ]:
buildSwappedPoemArray(langston_hughes_swapped,langston_hughes_poems_swapped)
buildSwappedPoemArray(mary_j_oliver_swapped,mary_j_oliver_poems_swapped)

In [ ]:
langston_hughes_swapped

In [ ]:
langston_hughes_swapped_to_json_file = langston_hughes_swapped.copy()
langston_hughes_swapped_to_json_file.pop('poems_swapped_array')
langston_hughes_swapped_to_json_file.pop('poem10')

mary_j_oliver_swapped_to_json_file = mary_j_oliver_swapped.copy()
mary_j_oliver_swapped_to_json_file.pop('poems_swapped_array')
mary_j_oliver_swapped_to_json_file.pop('poem10')



In [ ]:
langston_hughes_swapped_to_json_file


In [ ]:
mary_j_oliver_swapped_to_json_file

In [ ]:
with open('/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/Homework 9b/Datasets/langston_hughes_swapped_to_json_file_final.json', 'w') as outfile:
    json.dump(langston_hughes_swapped_to_json_file, outfile)

In [ ]:
with open('/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/Homework 9b/Datasets/mary_j_oliver_swapped_to_json_file_final.json', 'w') as outfile:
    json.dump(mary_j_oliver_swapped_to_json_file, outfile)

In [ ]:
with open('/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/Homework 9b/Datasets/langston_hughes_swapped_to_txt_file_final.txt', 'w') as outfile:
    json.dump(langston_hughes_swapped_to_json_file, outfile)

In [ ]:
with open('/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/Homework 9b/Datasets/mary_j_oliver_swapped_to_txt_file_final.txt', 'w') as outfile:
    json.dump(mary_j_oliver_swapped_to_json_file, outfile)

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import wordcloud
import nltk
import unicodedata
import contractions
import pickle
import urllib.request
import requests
import spacy
import json
%matplotlib inline

from bs4 import BeautifulSoup
from bs4.element import Comment
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from collections import defaultdict
from contractions import contractions_dict
from wordcloud import WordCloud, STOPWORDS
from transformers import pipeline

from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem.porter import PorterStemmer
from sentence_transformers import SentenceTransformer, util
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from spacy import displacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('punkt')

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from contractions import contractions_dict
from scipy import stats
from scipy import sparse

import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
scaler = MinMaxScaler()
std_scaler = StandardScaler()
stops = set(stopword_list)
stemmer = nltk.stem.SnowballStemmer('english')
wordlemmatizer = WordNetLemmatizer()
tokenizer = nltk.word_tokenize

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [ ]:
langston_hughes_choherence_input = langston_hughes.copy()
langston_hughes_choherence_input.pop('poems_array')
langston_hughes_choherence_input

In [ ]:
swapped_langston_hughes_choherence_input = langston_hughes_swapped.copy()
swapped_langston_hughes_choherence_input.pop('poems_swapped_array')
swapped_langston_hughes_choherence_input

In [ ]:
coherence_perplexity_calculator = mary_j_oliver.copy()
coherence_perplexity_calculator.pop('poems_array')
coherence_perplexity_calculator

In [ ]:
swapped_coherence_perplexity_calculator = mary_j_oliver_swapped.copy()
swapped_coherence_perplexity_calculator.pop('poems_swapped_array')
swapped_coherence_perplexity_calculator

In [ ]:
!pip install -upgrade gensim

In [ ]:
!pip install pyLDAvis

# Run Topic Modelling on combined text

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
def choherence_perplexity_calculator(poem_list):
  def sent_to_words(sentences):
      for sentence in sentences:
          yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

  data_words = list(sent_to_words(poem_list))

  print(data_words[:1])
  # Build the bigram and trigram models
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

  # Faster way to get a sentence clubbed as a trigram/bigram
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  # See trigram example
  print(trigram_mod[bigram_mod[data_words[0]]])
  # Define functions for stopwords, bigrams, trigrams and lemmatization
  def remove_stopwords(texts):
      return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

  def make_bigrams(texts):
      return [bigram_mod[doc] for doc in texts]

  def make_trigrams(texts):
      return [trigram_mod[bigram_mod[doc]] for doc in texts]

  def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
      """https://spacy.io/api/annotation"""
      texts_out = []
      for sent in texts:
          doc = nlp(" ".join(sent)) 
          texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
      return texts_out
  # Remove Stop Words
  #data_words_nostops = remove_stopwords(data_words)

  # Form Bigrams
  data_words_bigrams = make_bigrams(data_words)

  # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
  # python3 -m spacy download en
  nlp = spacy.load('en', disable=['parser', 'ner'])

  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN']) #, 'ADJ', 'VERB', 'ADV'

  print(data_lemmatized[:1])
  # Create Dictionary
  id2word = corpora.Dictionary(data_lemmatized)

  # Create Corpus
  texts = data_lemmatized

  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]

  # View
  print(corpus[:1])
 
  # lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
  #                                           id2word=id2word,
  #                                           num_topics=10, 
  #                                           random_state=100,
  #                                           update_every=1,
  #                                           chunksize=100,
  #                                           passes=10,
  #                                           alpha='auto',
  #                                           per_word_topics=True)
  lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
  print(lda_model.print_topics())
  # Visualize the topics
  pyLDAvis.enable_notebook()
  vis = gensimvis.prepare(lda_model, corpus, id2word)
  # vis
  # Compute Perplexity
  print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

  # Compute Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score: ', coherence_lda)
  return vis
  

In [ ]:
poem_count = len(langston_hughes_choherence_input)
langston_poem_list=[]
for j in langston_hughes_choherence_input:
  poem_index =  str(j)
  print(poem_index)
  print(langston_hughes_choherence_input[poem_index])
  langston_poem_list.append(langston_hughes_choherence_input[poem_index])
print(langston_poem_list)

# Original Langston Hughes Poem's perplexity and coherence score

In [ ]:
pip install attrs==19.1.0

In [ ]:
!pip install pandas --upgrade
# !pip install pandas==1.3.1

In [ ]:
vis = choherence_perplexity_calculator(langston_poem_list)

In [ ]:
vis

# **Love** is the Topic Name from based on the LDA model topics from Langston Hughes Poem's  (child, year, dream, night, sun)

In [ ]:
poem_count = len(coherence_perplexity_calculator)
mary_poem_list=[]
for j in coherence_perplexity_calculator:
  poem_index =  str(j)
  print(poem_index)
  print(coherence_perplexity_calculator[poem_index])
  mary_poem_list.append(coherence_perplexity_calculator[poem_index])
print(mary_poem_list)

# Original Mary J Oliver Poem's perplexity and coherence score

In [ ]:
vis = choherence_perplexity_calculator(mary_poem_list)

In [ ]:
vis

# **Love** is the Topic Name from based on the LDA model topics from Mary J Oliver Poem's  (dream, child, light, sum, year)

In [ ]:
swapped_langston_poem_list=[]
for j in swapped_langston_hughes_choherence_input:
  swapped_poem_index =  str(j)
  print(swapped_poem_index)
  print(swapped_langston_hughes_choherence_input[swapped_poem_index])
  swapped_langston_poem_list.append(swapped_langston_hughes_choherence_input[swapped_poem_index])
print(swapped_langston_poem_list)

# Swapped Langston Hughes Poem's perplexity and coherence score

In [ ]:
vis = choherence_perplexity_calculator(swapped_langston_poem_list)

In [ ]:
swapped_mary_poem_list=[]
for j in swapped_coherence_perplexity_calculator:
  swapped_poem_index =  str(j)
  print(swapped_poem_index)
  print(swapped_coherence_perplexity_calculator[swapped_poem_index])
  swapped_mary_poem_list.append(swapped_coherence_perplexity_calculator[swapped_poem_index])
print(swapped_mary_poem_list)

# Swapped Mary J Oliver Poem's perplexity and coherence score

In [ ]:
vis = choherence_perplexity_calculator(swapped_mary_poem_list)

In [ ]:
import numpy as np 
import pandas as pd

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import spacy

from scipy.spatial.distance import jensenshannon

import joblib

from IPython.display import HTML, display

from ipywidgets import interact, Layout, HBox, VBox, Box
import ipywidgets as widgets
from IPython.display import clear_output

from tqdm import tqdm
from os.path import isfile

import seaborn as sb
import matplotlib.pyplot as plt
plt.style.use("dark_background")

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 3000000


In [ ]:
def spacy_tokenizer(sentence):
    return [word.lemma_ for word in nlp(sentence) if not (word.like_num or word.is_stop or word.is_punct or word.is_space or len(word)==1)]

In [ ]:
vectorizer_langston = CountVectorizer(tokenizer = spacy_tokenizer, min_df=2)
vectorizer_mary = CountVectorizer(tokenizer = spacy_tokenizer, min_df=2)

data_vectorized_langston = vectorizer_langston.fit_transform(tqdm(langston_poem_list))
data_vectorized_mary = vectorizer_mary.fit_transform(tqdm(mary_poem_list))

# Combine poems, Summarize using tansformer and Save in file

In [ ]:
def combinePoems(poems, poet):
    poems_ref = []
    all_poems = []
    c = 0
    for i in poems:
      c += 1
      # print(i)
      all_poems.append(i)
      if c == 10:
        break
    p = {'all_poems':all_poems}
    poems_ref.append(p)
    po = {'poet': poet}
    poems_ref.append(po)
    return poems_ref

In [ ]:
langston_combined = swapped_langston_poem_list.copy()
langston_combined = combinePoems(langston_combined, 'Langston Hughes')
langston_combined

In [ ]:
mary_combined = swapped_mary_poem_list.copy()
mary_combined = combinePoems(mary_combined, 'Mary J Oliver')
mary_combined

In [ ]:
with open('/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/Homework 9b/Datasets/langston_hughes_swapped_combined_final.json', 'w') as outfile:
  json.dump(langston_combined, outfile)
with open('/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/Homework 9b/Datasets/mary_j_oliver_swapped_combined_final.json', 'w') as outfile:
  json.dump(mary_combined, outfile)

# Frequent words used in Langston Hughes poems

In [ ]:
word_count_langston = pd.DataFrame({'word': vectorizer_langston.get_feature_names(), 'count': np.asarray(data_vectorized_langston.sum(axis=0))[0]})

word_count_langston.sort_values('count', ascending=False).set_index('word')[:10].sort_values('count', ascending=True).plot(kind='barh')

# Frequent words used in Mary J Oliver poems

In [ ]:
word_count_mary = pd.DataFrame({'word': vectorizer_mary.get_feature_names(), 'count': np.asarray(data_vectorized_mary.sum(axis=0))[0]})

word_count_mary.sort_values('count', ascending=False).set_index('word')[:10].sort_values('count', ascending=True).plot(kind='barh')

#Name the Topics

In [ ]:
def topicModeling(text1):
  text1_list_orig = []
  bigram_text1 = []
  corpus_text1 = []

  text1_list_orig = [gensim.utils.simple_preprocess(text1)]
  bigram_text1 = gensim.models.Phrases(text1_list_orig) # higher threshold fewer phrases.
  trigram_text1 = gensim.models.Phrases(bigram_text1[text1_list_orig])
  #remove stop words
  remove_stop_text1 = remove_stopwords(text1_list_orig)
  # Form Bigrams
  data_words_bigrams_text1 = make_bigrams(remove_stop_text1, bigram_text1)
  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized_text1 = lemmatization(data_words_bigrams_text1, allowed_postags=['NOUN']) # , 'ADJ', 'VERB', 'ADV'
  id2word_text1 = corpora.Dictionary(data_lemmatized_text1)

  # Term Document Frequency
  corpus_text1 = [id2word_text1.doc2bow(text) for text in data_lemmatized_text1]
  lda_model_text1 = gensim.models.LdaMulticore(corpus=corpus_text1,
                                       id2word=id2word_text1,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
  doc_lda_text1 = lda_model_text1[corpus_text1]
  return lda_model_text1, corpus_text1, data_lemmatized_text1, id2word_text1



# **Conclusion**
Based on the frequency count of words, below are the top 3 topic names for both the poets:   
**Langston Hughes:** 
1. dream
2. educational
3. dark   
**Mary J Oliver:**
1. owner
2. provide
3. purpose

Based on the LDA weights, below are the top 3 topic names for both the poets:      
**Langston Hughes:** 
1. little 
2. purpose 
3. letter  

**Mary J Oliver:**
1. dream
2. child
3. light

# References


1. https://github.com/aarsanjani/nlp-poets
2. https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
3. https://sjsu.instructure.com/courses/1465859/assignments/5992374
